In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent, Runner, SQLiteSession, InputGuardrail, GuardrailFunctionOutput
from agents.exceptions import InputGuardrailTripwireTriggered
from pydantic import BaseModel, Field
from loguru import logger
from IPython.display import display, Markdown

load_dotenv(override=True)
session = SQLiteSession("conversations")

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    logger.error("OPENAI_API_KEY is not set in the environment variables.")
else:
    logger.info(f"OpenAI API exists: {openai_api_key[:10]}...")

2025-08-07 12:24:47.637 | INFO     | __main__:<module>:18 - OpenAI API exists: sk-proj-Yq...


In [2]:
model = "gpt-4o-mini"
openai_client = OpenAI(api_key=openai_api_key)

In [3]:
agent_one = Agent(
    name="Bangladesh knowledge Agent",
    handoff_description="This agent is designed to answer questions about Bangladesh.",
    model=model,
    instructions="""
    You are an expert on Bangladesh. Answer questions about its history, culture, geography, and current events. Provide detailed and accurate information.""",
)

In [4]:
agent_two = Agent(
    name="Norway knowledge Agent",
    handoff_description="This agent is designed to answer questions about Norway.",
    model=model,
    instructions="""
    You are an expert on Norway. Answer questions about its history, culture, geography, and
    current events. Provide detailed and accurate information.""",
)

In [5]:
agent_three = Agent(
    name="Scotland knowledge Agent",
    handoff_description="This agent is designed to answer questions about Scotland.",
    model=model,
    instructions="""
    You are an expert on Scotland. Answer questions about its history, culture, geography, and
    current events. Provide detailed and accurate information.""",
)

In [6]:
class CountryOutput(BaseModel):
    is_country: bool
    reasoning: str

In [7]:
guardrail_agent = Agent(
    name="Guardrail Agent",
    model=model,
    instructions="""
    You are a guardrail agent that determines if a question is related to Bangladesh, Norway, or Scotland.

    Analyze the input question and determine if it's related to Bangladesh, Norway, or Scotland in ANY way, including:
    - Direct mentions of the countries
    - Cities, landmarks, or geographical features within these countries
    - Historical events, people, or cultural aspects from these countries
    - Any topic that would require knowledge about Bangladesh, Norway, or Scotland to answer

    Examples of questions that should return true:
    - "What is the capital of Bangladesh?" (directly about Bangladesh)
    - "Where is Dhaka?" (Dhaka is in Bangladesh)
    - "Tell me about Bergen" (Bergen is in Norway)
    - "What is Edinburgh Castle?" (Edinburgh is in Scotland)

    Respond in this exact JSON format:
    {
        "is_country": true/false,
        "reasoning": "your explanation here"
    }

    Set is_country to true if the question relates to Bangladesh, Norway, or Scotland.
    Set is_country to false only if the question has no connection to these three countries.
    """,
)

In [8]:
async def country_guardrail(ctx, agent, input_data):
    try:
        result = await Runner.run(guardrail_agent, input_data, context=ctx.context if hasattr(ctx, "context") else None)

        # Parse the JSON response
        response_text = result.final_output.strip()
        if response_text.startswith('```json'):
            response_text = response_text.replace('```json', '').replace('```', '').strip()

        parsed_response = json.loads(response_text)
        final_output = CountryOutput(
            is_country=parsed_response["is_country"],
            reasoning=parsed_response["reasoning"]
        )

        return GuardrailFunctionOutput(
            output_info=final_output,
            tripwire_triggered=not final_output.is_country,
        )
    except Exception as e:
        logger.error(f"Guardrail error: {e}")
        return GuardrailFunctionOutput(
            output_info=CountryOutput(is_country=False, reasoning="Error in guardrail processing"),
            tripwire_triggered=True,
        )

In [9]:
triage_agent = Agent(
    name="All Countries Triage Agent",
    handoff_description="This agent triages questions to the appropriate country-specific agent.",
    instructions="You determine which country-specific agent should handle the question based on its content. ",
    handoffs=[
        agent_one,
        agent_two,
        agent_three,
    ],
    input_guardrails=[InputGuardrail(guardrail_function=country_guardrail)],
)

In [14]:
async def ask(query: str):
    try:
        question = query or "What is the capital of Bangladesh?"
        display(Markdown(f"**Question:** {question}"))

        answer = await Runner.run(
            triage_agent,
            question,
        )
        if "#" not in answer.final_output:
            print(f"Answer: {answer.final_output}")
        else:
            display(Markdown(f"Answer: {answer.final_output}"))
    except InputGuardrailTripwireTriggered as e:
        logger.error(f"Input guardrail triggered: {e}")
    except Exception as e:
        logger.error(f"An error occurred: {e}")

await ask("where is dhaka?")

**Question:** where is dhaka?

Answer: Dhaka is the capital city of Bangladesh, located in the central part of the country along the banks of the Buriganga River. It is one of the most densely populated cities in the world and serves as the political, economic, and cultural heart of Bangladesh. Dhaka is situated in the Dhaka Division, which is one of the administrative regions of the country. The city is known for its rich history, vibrant culture, and rapid urban growth.


In [15]:
await ask("Tell me about Norway Culture and geographic beauty")

**Question:** Tell me about Norway Culture and geographic beauty

Answer: ### Culture of Norway

**1. Traditional Heritage:**
   - **Viking Influence:** Norway's culture has deep roots in its Viking heritage, with Norse mythology playing a significant role in art, literature, and local traditions.
   - **Folklore:** Traditional stories often feature trolls, elves, and other creatures from Nordic mythology, reflecting a rich oral storytelling tradition.

**2. Arts:**
   - **Literature:** Norway has produced notable writers, such as Henrik Ibsen and Knut Hamsun, known for their contributions to modern drama and literature.
   - **Visual Arts:** Important artists include Edvard Munch, famous for his painting "The Scream," which explores themes of existential angst.
   - **Music:** Norway is known for its folk music, with traditional instruments like the Hardanger fiddle, as well as a vibrant contemporary music scene spanning genres from pop to black metal.

**3. Festivals and Celebrations:**
   - **National Day (17th May):** Celebrated with parades, children dressing in traditional costumes (bunad), and various local festivities.
   - **Cultural Events:** Festivals like the Bergen Fest and Oslo Jazz Festival celebrate music and arts, drawing both local and international talent.

**4. Cuisine:**
   - Traditional dishes include dishes like **lutefisk**, **rakfisk**, and fresh seafood such as salmon and herring. A focus on locally sourced ingredients is growing, alongside a strong emphasis on sustainability.

### Geographic Beauty of Norway

**1. Fjords:**
   - Norway is renowned for its dramatic fjords, such as the Geirangerfjord and Nærøyfjord, characterized by steep cliffs, cascading waterfalls, and serene waters. These UNESCO World Heritage sites attract visitors worldwide.

**2. Mountains:**
   - The **Jotunheimen National Park** is famous for its rugged mountains and hiking trails, including Galdhøpiggen, the highest peak in Northern Europe.

**3. Northern Lights:**
   - The country offers breathtaking views of the Aurora Borealis, particularly in northern regions like Tromsø during winter months.

**4. Coastline and Islands:**
   - Norway has one of the longest coastlines in the world, dotted with picturesque islands. The Lofoten Islands are notable for their dramatic scenery, traditional fishing villages, and outdoor activities.

**5. Wildlife:**
   - The diverse ecosystems host a range of wildlife, including reindeer, polar bears in Svalbard, and various marine life, making Norway a prime location for nature enthusiasts.

**6. Urban and Rural Landscapes:**
   - Cities like Oslo, Bergen, and Trondheim blend modern architecture with historical buildings. Additionally, quaint villages and rural regions reflect Norway's agricultural roots and scenic beauty.

### Conclusion

Norway's rich cultural heritage and stunning natural landscapes make it a unique country with much to offer in terms of history, art, and outdoor experiences. Whether exploring its vast fjords or engaging with its vibrant cultural scene, Norway presents a harmonious blend of tradition and modernity.

In [16]:
await ask("Can a Bangladeshi citizen travel to Norway? can he survive long time in Norway? can he adapt the cold weather?")

**Question:** Can a Bangladeshi citizen travel to Norway? can he survive long time in Norway? can he adapt the cold weather?

Answer: Yes, a Bangladeshi citizen can travel to Norway, but they need to obtain a visa before making their trip. The type of visa required will depend on the purpose of the visit, such as tourism, work, or study. Typically, a Schengen visa would be needed for short stays.

### Survival and Adaptation

1. **Surviving in Norway:**
   - The cost of living in Norway is high, so a good understanding of the financial requirements is essential. 
   - Securing employment or having sufficient funds is crucial for long-term stay.

2. **Adapting to Cold Weather:**
   - Norway experiences harsh winters with temperatures often dropping below freezing. Adapting to this climate requires proper clothing and lifestyle adjustments:
     - **Clothing:** Investing in thermal clothing, waterproof jackets, and insulated footwear is important.
     - **Lifestyle:** Adapting to indoor heating and shorter daylight hours in winter can also be beneficial.
   - Many migrants successfully adapt to the cold weather, especially with proper preparation and mindset.

In summary, while a Bangladeshi citizen can travel to Norway and can adapt to the cold weather, it is important to prepare adequately for both the climate and the cultural adjustments required.

In [12]:
await ask("What is the capital of India?")

**Question:** What is the capital of India?

2025-08-07 12:34:09.541 | ERROR    | __main__:ask:12 - Input guardrail triggered: Guardrail InputGuardrail triggered tripwire


In [18]:
await ask("Where is Nepal?")

**Question:** Where is Nepal?

2025-08-07 12:43:06.173 | ERROR    | __main__:ask:15 - Input guardrail triggered: Guardrail InputGuardrail triggered tripwire


In [17]:
await ask("What is the capital of Scotland?")

**Question:** What is the capital of Scotland?

Answer: The capital of Scotland is Edinburgh. It is known for its historic and architectural landmarks, including the Edinburgh Castle, the Palace of Holyroodhouse, and the annual Edinburgh Festival, which encompasses various arts festivals held throughout August. The city is also renowned for its vibrant cultural scene and beautiful landscapes, particularly the nearby Arthur's Seat.
